### **INCREMENTAL DATA INGESTION**

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.raw.sliver;
CREATE VOLUME IF NOT EXISTS workspace.raw.gold;
CREATE VOLUME IF NOT EXISTS workspace.raw.bronze;
CREATE VOLUME IF NOT EXISTS workspace.silver.silvervolume;
CREATE VOLUME IF NOT EXISTS workspace.gold.goldvolume;
CREATE VOLUME IF NOT EXISTS workspace.bronze.bronzevolume

In [0]:
dbutils.widgets.text("src", "")
src_value = dbutils.widgets.get("src")

In [0]:
df = spark.readStream.format("cloudFiles")\
          .option("cloudFiles.format", "csv")\
          .option("cloudFiles.schemaLocation", f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoint")\
          .option("cloudFile.shemaEvolutionMode", "rescue")\
          .load(f"/Volumes/workspace/raw/rawvolume/rawdata/{src_value}/")

In [0]:
 df.writeStream.format("delta")\
        .outputMode("append")\
        .trigger(once=True)\
        .option("checkpointLocation", f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoint")\
        .option("path", f"/Volumes/workspace/bronze/bronzevolume/{src_value}/data")\
        .start()

In [0]:
%sql
select * from delta.`/Volumes/workspace/bronze/bronzevolume/bookings/data/`